# Transacciones,
# Entradas y Salidas,
# y Emisión

<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/06_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [ ]:
! pip install eciespy eth_keys cryptos qrcode base58 
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git

# Crear Transacciones de Criptomoneda (Create Cryptocurrency Transactions)

## Esquema (Outline) 
Esta será una introducción a las transacciones de criptomonedas. Los participantes se conectan a la red del Colegio Invisible y publican su dirección criptográfica para recibir monedas. La transacción de financiación se construye, firma y transmite. Una vez que se confirma la transacción de financiación, los participantes devuelven las monedas construyendo, firmando y transmitiendo.
***
This will be an introduction to cryptocurrency transactions. Participants connect to the Invisible College network and post their crypto address to receive coins. The funding transaction is constructed, signed and broadcast. Once the funding transaction confirms, participants return the coins by constructing, signing, and broadcasting.
 
## Metas (Goals)
***
***
By the end of this notebook you should be able to:
***
Al final de este cuaderno, usted debería ser capaz de:
***
***
- Comprender la estructura de una transacción de Bitcoin/Dogecoin\
(Understand the structure of a Bitcoin/Dogecoin transaction)
- Importar las claves\
(Import Keys)
- Conectarse a Google Sheets compartidas\
(Connect to shared Google Sheets)
- Publicar la dirección\
(Post address)
- Consultar las otras direcciones\
(Query the addresses)
- Recibir criptomonedas\
(Receive crypto coins)
- Comprender UTXO las salidas de transacciones no gastadas\
(Understand UTXO Unspent Transaction Outputs)
- Comprender las entradas y salidas de transacciones\
(Understand Transaction Inputs and Outputs)
- Construir la transacción\
(Construct the transaction)
- Firmar transacción\
(Sign transaction)
- Convertir transacción a hexadecimal\
(Convert transaction to hex)
- Transmitir la transacción\
(Broadcast transaction)
- Supervisar la red para la confirmación\
(Monitor the network for confirmation)

# Importaciones y Definiciones (Imports and Definitions)

Crear funciones para importar claves y direcciones de criptomonedas.
***
Create functions for importing keys and cryptocurrency addresses.

In [ ]:
import hashlib
import os
import getpass
import ecies
import eth_keys
import cryptos
import qrcode
import base58
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def import_privKey(privkey_filepath,password=None):
    if password==None:
        password = getpass.getpass("Input password for decrypting keyfile: ")
    password=password.encode()
    f=open(privkey_filepath,'rb')
    decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
    privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
    return privKey

def import_pubKey(pubkey_filepath):
    f=open(pubkey_filepath,'rb')
    privKey=eth_keys.keys.PublicKey(f.read())
    return privKey

def import_addr(addr_filepath):
    return open(addr_filepath,'rb').read().decode()

# Comenzar con Claves Existentes (Start with Existing Keys)

Acceder a las claves generadas en el cuaderno 5
***
Access the keys generated in the cuaderno 5

## Comenzar con Claves en Google Drive
## (Start with Keys on Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip

## Comenzar con Claves en la Disco Local
## (Start with Keys on Local Drive)

In [ ]:
#from google.colab import files
#fu=files.upload();

In [ ]:
#! unzip llaves.zip

In [ ]:
#! ls llaves

# Import Private Key and Address

In [ ]:
privKey=import_privKey('llaves/mi_prv.enc','')
addr=import_addr('llaves/mi_addr.bin')

In [ ]:
addr

In [ ]:
from IPython.display import Image
qr=Image('llaves/mi_addr.png')
display(qr)

# Connect to Colegio Message Spreadsheet

## Decrypt Credentials

$\color{white}{\text{c0leg101nv1s1ble}}$

Use la contraseña de arriba para descifrar las credenciales
***
Use password above to decrypt credentials


In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

## Autenticar y Conectarse a la Google Sheet de Mensajes (Authenticate and Connect to Message Google Sheet)

Autenticarse usando credenciales descifradas para la API de Google Drive. Use la biblioteca `gspread` para leer y escribir en una hoja de cálculo compartida.
***
Authenticate using decrypted credentials for Google Drive API. Use `gspread` library to read and write to shared spreadsheet.

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# Leer y Escribir en la Google Sheet de Mensajes (Read and Write to Messages Google Sheet)

## Leer Google Sheet (Read Google Sheet)

In [ ]:
sheets.worksheets()

In [ ]:
sheet4=sheets.worksheets()[4]
sheet4.row_values(1)

In [ ]:
#sheet4.clear()
#sheet4.append_row(["name", "address"])

## Ingresar el Nombre (Input Name)

In [ ]:
my_name=input('input name')

## Leer Nombres Publicados (Read Posted Names)

In [ ]:
peeps=sheet4.col_values(1)[1:]
peeps

## Escribir en la Google Sheet para Publicar Nombre (Write to Google Sheet to Publish Name)

Si el nombre está en la lista, escriba la dirección en la celda correspondiente de la hoja\
(If the name is in the list then write address to the appropriate cell of sheet)\
Si el nombre no está presente, agregue el nombre y la dirección a la hoja\
(If name is not present append name and address to sheet)

In [ ]:
if not (my_name in peeps) :
  sheet4.append_row([my_name,addr])
else:
  sheet4.update_cell(row=peeps.index(my_name)+2,col=2,value=addr)

## Leer Direcciones (Read Addresses)

In [ ]:
import pandas as pd

dataframe4 = pd.DataFrame(sheet4.get_all_records())
dataframe4

## Construir Lista de los Direcciones de Destinatario (Construct List of Recipient Addresses) 

In [ ]:
addresses=dataframe4.address.tolist()
addresses

# Enviar Direcciones de Criptomonedas con Transacción de Financiación (Send Addresses Cryptocurrency with Funding Transaction)

## Importar 'cryptos' y Crear un Objeto de Dogecoin (Import `cryptos` and Create Dogecoin Object)

In [ ]:
import cryptos

In [ ]:
doge=cryptos.Doge()

## Las Entradas (Inputs)

### Identificar la Dirección de Financiación (Identify Funding Address)

In [ ]:
banco=addresses[0]
banco

### Determinar las Entradas de UTXO (Determine UTXO Inputs)

***
***
Los UTXO son salidas de transacciones no gastadas. Comenzamos revisando la cadena de bloques para identificar las monedas que la dirección de financiación ha recibido pero que aún no ha gastado. Estos son UTXO.
***
UTXOs are Unspent Transaction Outputs. We start by checking the blockchain to identify coins the funding address has received but has not yet spent. These are UTXOs. 
***
***

Cada UTXO contiene (each UTXO contains):
- valor (value)
- transacción (transaction)
  - hash (hash)
  - índice (index)

In [ ]:
#UTXOS
unspents=doge.unspent(banco)
unspents

### Satoshis

El valor se informa en unidades de Satoshis/Dogetoshis. Un Satoshi es un $\frac{1}{100,000,000}$ de una moneda.
***
The value is reported in units of Satoshis/Dogetoshis. A Satoshi is $\frac{1}{100,000,000}$ of a coin.

### Sumar Salidas No Gastadas (Sum Unspent Outputs)

In [ ]:
unspent_total=sum([unspent['value'] for unspent in unspents])
unspent_total

## Salidas (Outputs)

### Calcular Valores de Salida (Calculate Output Values)

- `gift`: es la cantidad entregada a cada participante\
(the amount given to each participant)
- `gift_total`: la cantidad total entregada\
(the total amount given)
- `mining_fee`: cantidad entregada a los mineros\
(amount given to miners)
- `change`: cantidad sobrante que el remitente enviará a una dirección de cambio\
(the left over amount that the sender will send to a change address)

In [ ]:
gift=10_00_000_000
gift_total = gift * len(addresses)
mining_fee = 2_00_000_000
change = unspent_total - gift_total - mining_fee 

### Construct Output List

Create outputs including:
- `gift` regalo a cada participante\
 `gift` to each participant
- `change` enviado de vuelta a `banco` direccion\
`change` sent back to `banco` address

In [ ]:
outputs=[{'value': gift, 'address': address} for address in addresses]
outputs.append({'value': change, 'address': banco})
outputs


## Make Funding Transaction

In [ ]:
fund_txn=doge.mktx(unspents,outputs )
fund_txn

## Sign Funding Transaction

In [ ]:
doge.signall(fund_txn,privKey.to_hex()[2:])

In [ ]:
fund_txn

In [ ]:
fund_txn_hex=cryptos.serialize(fund_txn)
fund_txn_hex

## Check kilobyte Size of Transaction

In [ ]:
kb=len(fund_txn_hex)/2000
kb

In [ ]:
mining_fee/1E8/kb

## Broadcast Transaction

In [ ]:
broadcast_fund_txn=doge.pushtx(fund_txn_hex)
broadcast_fund_txn

## Inspect Transaction

In [ ]:
inspect_fund_txn=doge.fetchtx( broadcast_fund_txn['data']['txid'])
inspect_fund_txn

# Return Coins

## Construct Return Transaction

### Inputs

In [ ]:
mi_unspents=doge.unspent(addr)
mi_unspents


In [ ]:
mi_unspent_total=sum([unspent['value'] for unspent in mi_unspents])
mi_unspent_total

### Outputs

In [ ]:
mi_mining_fee = 1_00_000_000
disponible = mi_unspent_total - mi_mining_fee
disponible

In [ ]:
destinos=[{'value': disponible, 'address': banco} ]
destinos

### Constuction

In [ ]:
volver_txn=doge.mktx(mi_unspents, destinos)
volver_txn

## Signing and Serializing

In [ ]:
doge.signall(volver_txn,privKey.to_hex()[2:])

In [ ]:
volver_txn_hex=cryptos.serialize(volver_txn)
volver_txn_hex

## Broadcasting

In [ ]:
broadcast_volver_txn=doge.pushtx(volver_txn_hex)
broadcast_volver_txn

## Monitoring Network

In [ ]:
inspect_volver_txn=doge.fetchtx( broadcast_volver_txn['data']['txid'])
inspect_volver_txn